In [1]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Error loading punkt: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1076)>
[nltk_data] Error loading stopwords: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1076)>
[nltk_data] Error loading wordnet: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1076)>


False

In [2]:
import pandas as pd
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from pandas_profiling import ProfileReport
import string
import json
import pickle

In [3]:
# Need to cleanup and tokenize words first
users_df = pd.read_csv(r'./Scraping/data/uber_users.csv') # Per user data
uber_df = pd.read_pickle(r'./Scraping/data/uber.p')

In [4]:
users_df.drop(users_df.columns[0], axis=1, inplace=True)
uber_df.columns = ['source', 'source_id', 'target', 'target_id', 'type_of_content', 'type_of_content_2', 'created_at', 'all_data']
uber_df.drop(columns=['type_of_content_2'], inplace=True)

In [5]:
uber_df.index

RangeIndex(start=0, stop=5568, step=1)

In [6]:
uber_df['all_data'] = uber_df.apply(lambda row: json.loads(json.dumps(row['all_data']._json)), axis=1)

In [7]:
uber_df['text'] = uber_df.apply(lambda row: row['all_data']['text'], axis=1)

In [8]:
uber_df

,source,source_id,target,target_id,type_of_content,created_at,all_data,text
0,Uber_Support,2815754953,wordofbrandon,2566081240,Reply,2019-08-31 10:53:53,{'created_at': 'Sat Aug 31 10:53:53 +0000 2019...,@wordofbrandon Thanks for bringing this to our...
1,Uber_Support,2815754953,wordofbrandon,2566081240,Reply,2019-08-31 10:53:53,{'created_at': 'Sat Aug 31 10:53:53 +0000 2019...,@wordofbrandon Thanks for bringing this to our...
2,Uber_Support,2815754953,CapeVgirl,719534538,Reply,2019-10-01 07:45:30,{'created_at': 'Tue Oct 01 07:45:30 +0000 2019...,@CapeVgirl Here to help. We suggest restarting...
3,Uber_Support,2815754953,CapeVgirl,719534538,Reply,2019-10-01 07:45:30,{'created_at': 'Tue Oct 01 07:45:30 +0000 2019...,@CapeVgirl Here to help. We suggest restarting...
4,SatishInnov8,722043526461145088,Uber_Support,2815754953,Reply,2019-12-09 17:11:51,{'created_at': 'Mon Dec 09 17:11:51 +0000 2019...,@Uber_Support \n1/n\nI wish to respectfully st...
...,...,...,...,...,...,...,...,...
5563,ajb_neuk,1344742630731747328,Uber_Support,2815754953,Reply,2021-03-31 17:15:48,{'created_at': 'Wed Mar 31 17:15:48 +0000 2021...,@Uber_Support Yeah cause that link is about as...
5564,AdrianMatache7,1376698104028794882,Uber_Support,2815754953,Reply,2021-03-31 17:18:34,{'created_at': 'Wed Mar 31 17:18:34 +0000 2021...,@Uber_Support Hi guys. How long takes for a ba...
5565,k_sovs,1369169148580823040,Uber_Support,2815754953,Reply,2021-03-31 17:19:01,{'created_at': 'Wed Mar 31 17:19:01 +0000 2021...,@Uber_Support I filled out the form many times...
5566,MY0U1SANA,918655647763324928,Uber_Support,2815754953,Reply,2021-03-31 17:22:01,{'created_at': 'Wed Mar 31 17:22:01 +0000 2021...,@Uber_Support @fltrtae hold on


In [9]:
uber_df.drop_duplicates(subset=['source', 'target', 'text', 'type_of_content', 'created_at'])

,source,source_id,target,target_id,type_of_content,created_at,all_data,text
0,Uber_Support,2815754953,wordofbrandon,2566081240,Reply,2019-08-31 10:53:53,{'created_at': 'Sat Aug 31 10:53:53 +0000 2019...,@wordofbrandon Thanks for bringing this to our...
2,Uber_Support,2815754953,CapeVgirl,719534538,Reply,2019-10-01 07:45:30,{'created_at': 'Tue Oct 01 07:45:30 +0000 2019...,@CapeVgirl Here to help. We suggest restarting...
4,SatishInnov8,722043526461145088,Uber_Support,2815754953,Reply,2019-12-09 17:11:51,{'created_at': 'Mon Dec 09 17:11:51 +0000 2019...,@Uber_Support \n1/n\nI wish to respectfully st...
5,Uber_Support,2815754953,realchetty,28766827,Reply,2019-12-20 12:02:54,{'created_at': 'Fri Dec 20 12:02:54 +0000 2019...,@realchetty We definitely want to ensure that ...
6,Uber_Support,2815754953,getfitwithgiddy,340722032,Reply,2019-12-22 11:25:56,{'created_at': 'Sun Dec 22 11:25:56 +0000 2019...,@getfitwithgiddy We definitely want to follow ...
...,...,...,...,...,...,...,...,...
5563,ajb_neuk,1344742630731747328,Uber_Support,2815754953,Reply,2021-03-31 17:15:48,{'created_at': 'Wed Mar 31 17:15:48 +0000 2021...,@Uber_Support Yeah cause that link is about as...
5564,AdrianMatache7,1376698104028794882,Uber_Support,2815754953,Reply,2021-03-31 17:18:34,{'created_at': 'Wed Mar 31 17:18:34 +0000 2021...,@Uber_Support Hi guys. How long takes for a ba...
5565,k_sovs,1369169148580823040,Uber_Support,2815754953,Reply,2021-03-31 17:19:01,{'created_at': 'Wed Mar 31 17:19:01 +0000 2021...,@Uber_Support I filled out the form many times...
5566,MY0U1SANA,918655647763324928,Uber_Support,2815754953,Reply,2021-03-31 17:22:01,{'created_at': 'Wed Mar 31 17:22:01 +0000 2021...,@Uber_Support @fltrtae hold on


In [10]:
ProfileReport(uber_df, minimal=True, title='Uber Support Twitter Data')

Summarize dataset:   0%|          | 0/16 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

In [11]:
uber_df.columns

Index(['source', 'source_id', 'target', 'target_id', 'type_of_content',
       'created_at', 'all_data', 'text'],
      dtype='object')

In [68]:
def TokenizeProcess(df):
    word_lemmatizer = WordNetLemmatizer()

    df_tk = pd.DataFrame(columns=['source', 'source_id', 'target', 'target_id', 'type_of_content', 'created_at', 'all_data', 'text'])

    for (source, source_id, target, target_id, type_of_content, created_at, all_data, text) in df.values.tolist():
        # Tokenize
        tokens = word_tokenize(str(text).lower().replace("'",""))

        # Strip punctuation
        punctuation_list = str.maketrans('', '', string.punctuation)
        tokens_strp = [w.translate(punctuation_list) for w in tokens]

        # Remove other non-alphabetic tokens
        words = [word for word in tokens_strp if word.isalpha()]

        # Stop words
        stop_words = set(stopwords.words('english'))
        words = [w for w in words if not w in stop_words]

        # Lemmatize
        # Note: We might need to do this for the brands we want to detect as well
        words_lm = [word_lemmatizer.lemmatize(w) for w in words]
        df_tk = df_tk.append({'source': source, 'source_id': source_id, 'target': target, 'target_id': target_id, 'text': words_lm, 'type_of_content': type_of_content, 'created_at': created_at, 'all_data': all_data} ,ignore_index=True) 
    return df_tk
def TokenizeProcess_handles(df,tokenized_handles):
    word_lemmatizer = WordNetLemmatizer()

    df_tk = pd.DataFrame(columns=['source', 'source_id', 'target', 'target_id', 'type_of_content', 'created_at', 'all_data', 'text'])

    for (source, source_id, target, target_id, type_of_content, created_at, all_data, text) in df.values.tolist():
        # Tokenize Source
#         tokens = word_tokenize(str(target).lower().replace("'",""))

#         # Strip punctuation
#         punctuation_list = str.maketrans('', '', string.punctuation)
#         tokens_strp = [w.translate(punctuation_list) for w in tokens]

#         # Remove other non-alphabetic tokens
#         words = [word for word in tokens_strp if word.isalpha()]

#         # Stop words
#         stop_words = set(stopwords.words('english'))
#         words = [w for w in words if not w in stop_words]
#         words_lm_target = words
        
        
        
#         # Tokenize Target
#         tokens = word_tokenize(str(source).lower().replace("'",""))

#         # Strip punctuation
#         punctuation_list = str.maketrans('', '', string.punctuation)
#         tokens_strp = [w.translate(punctuation_list) for w in tokens]

#         # Remove other non-alphabetic tokens
#         words = [word for word in tokens_strp if word.isalpha()]

#         # Stop words
#         stop_words = set(stopwords.words('english'))
#         words = [w for w in words if not w in stop_words]
#         words_lm_source = words
        
        
        tokens = word_tokenize(str(text).lower().replace("'",""))
        
        # Strip punctuation
        punctuation_list = str.maketrans('', '', string.punctuation)
        tokens_strp = [w.translate(punctuation_list) for w in tokens]

#         print(words_lm_source)
#         print(words_lm_target)
        # Remove other non-alphabetic tokens
        words = [word for word in tokens_strp if word.isalpha() and word not in tokenized_handles]#(word not in words_lm_source and word not in words_lm_target)]
        # Stop words
        stop_words = set(stopwords.words('english'))
        words = [w for w in words if not w in stop_words]
        
        # Lemmatize
        # Note: We might need to do this for the brands we want to detect as well
        words_lm = [word_lemmatizer.lemmatize(w) for w in words]

        print(words_lm)
        df_tk = df_tk.append({'source': source, 'source_id': source_id, 'target': target, 'target_id': target_id, 'text': words_lm, 'type_of_content': type_of_content, 'created_at': created_at, 'all_data': all_data} ,ignore_index=True) 
    return df_tk



In [14]:
#uber_df_tk = TokenizeProcess(uber_df)

In [69]:
#The handles change once tokenized. Therefore we will change them to match the tokenization
#None of the handles get turned into more than one element due to the nature of twitter handles
handles=list(set(list(uber_df['source'])+list(uber_df['target'])))
tokenized_handles=[]
for handle in handles:
    tokens = word_tokenize(str(handle).lower().replace("'",""))

    # Strip punctuation
    punctuation_list = str.maketrans('', '', string.punctuation)
    tokens_strp = [w.translate(punctuation_list) for w in tokens]

    # Remove other non-alphabetic tokens
    words = [word for word in tokens_strp if word.isalpha()]

    # Stop words
    stop_words = set(stopwords.words('english'))
    words = [w for w in words if not w in stop_words]
    words_lm = words
    
    
    tokenized_handles.append(words)
tokenized_handles = [x[0] for x in tokenized_handles if len(x)>0]

In [72]:
uber_df_tk = TokenizeProcess_handles(uber_df,handles)

['thanks', 'bringing', 'attention', 'team', 'currently', 'reviewing', 'http']
['thanks', 'bringing', 'attention', 'team', 'currently', 'reviewing', 'http']
['help', 'suggest', 'restarting', 'app', 'work', 'reset', 'network', 'setting', 'http']
['help', 'suggest', 'restarting', 'app', 'work', 'reset', 'network', 'setting', 'http']
['wish', 'respectfully', 'state', 'regular', 'user', 'using', 'e', 'mail', 'id', 'http']
['definitely', 'want', 'ensure', 'addressed', 'appropriately', 'issue', 'raised', 'want', 'http']
['definitely', 'want', 'follow', 'please', 'send', 'u', 'dm', 'phone', 'number', 'http']
['definitely', 'want', 'follow', 'please', 'send', 'u', 'dm', 'phone', 'number', 'http']
['definitely', 'want', 'get', 'sorted', 'order', 'best', 'assist', 'http']
['definitely', 'want', 'get', 'sorted', 'order', 'best', 'assist', 'http']
['thank', 'providing', 'info', 'order', 'protect', 'privacy', 'please', 'delete', 'http']
['thank', 'providing', 'info', 'order', 'protect', 'privacy', '

['definitely', 'want', 'get', 'sorted', 'uk', 'concern', 'please', 'send', 'u', 'note', 'http']
['hell', 'difficult', 'reach', 'trying', 'call']
['would', 'recommend', 'send', 'note', 'registered', 'driver', 'advising', 'http']
['would', 'recommend', 'send', 'note', 'registered', 'driver', 'advising', 'http']
['app', 'glitched', 'ordering', 'food', 'order', 'duplicated', 'please', 'help', 'want', 'http', 'tcoaoarvesvtt']
['hello', 'past', 'day', 'dashboard', 'allowing', 'add', 'item']
['help', 'sent', 'dm', 'kindly', 'check']
['help', 'sent', 'dm', 'kindly', 'check']
['ive', 'seen', 'youve', 'returned', 'driver', 'stole', 'intention', 'using', 'cowboy', 'http']
['charged', 'cancellation', 'charge', 'able', 'cancel', 'app', 'working', 'http']
['trip', 'jn', 'deivers', 'way', 'force', 'passenger', 'cancel', 'trip']
['plwase', 'refund', 'cancellation', 'charge', 'account']
['help', 'last', 'ride', 'app', 'helping']
['told', 'wait', 'day', 'refund', 'overcharged', 'trip', 'yesterday', 'bes

['requested', 'ride', 'airport', 'rio', 'de', 'janeiro', 'driver', 'took', 'different', 'route', 'added', 'http']
['please', 'check', 'dm', 'respond', 'thanks']
['omg', 'drive', 'took', 'absolute', 'worst', 'route', 'texan', 'game', 'walked', 'get', 'away', 'http']
['many', 'time', 'tell', 'pay', 'charge', 'locked', 'damned', 'account']
['hi', 'need', 'help', 'lost', 'phone', 'app', 'preventing', 'reaching', 'driver']
['hi', 'need', 'help', 'lost', 'phone', 'app', 'preventing', 'reaching', 'driver']
['order', 'found', 'delivery', 'partner', 'minute', 'cancel', 'order', 'without', 'getting', 'http']
['hello', 'need', 'help', 'moved', 'new', 'zealand', 'please', 'help']
['cant', 'find', 'way', 'contact', 'issue', 'isnt', 'listed', 'automated', 'help', 'list', 'http']
['employ', 'driver', 'purposely', 'choose', 'connect', 'rider', 'agreed', 'pick']
['hey', 'guy', 'drive', 'refuse', 'cancel', 'trip', 'charged', 'cancellation', 'fee', 'http']
['gon', 'na', 'respond', 'query', 'without', 'co

['told', 'fraudulent', 'cleaning', 'fee', 'talked', 'http']
['keep', 'receiving', 'text', 'code', 'requested', 'dont', 'know', 'http']
['definitely', 'want', 'get', 'sorted', 'understand', 'want', 'issue', 'resolved', 'http']
['thank', 'please', 'let', 'know', 'need', 'anything']
['book', 'trip', 'ahead', 'anymore', 'trying', 'book', 'one', 'tomorrow']
['shame', 'ignore', 'customer', 'say', 'issue', 'response', 'would', 'nice']
['feel', 'like', 'major', 'news', 'outlet', 'need', 'know', 'http']
['feel', 'like', 'major', 'news', 'outlet', 'need', 'know', 'http']
['feel', 'like', 'major', 'news', 'outlet', 'need', 'know', 'http']
['feel', 'like', 'major', 'news', 'outlet', 'need', 'know', 'http']
['feel', 'like', 'major', 'news', 'outlet', 'need', 'know', 'http']
['made', 'trip', 'december', 'well', 'day', 'ago', 'gotten', 'paid', 'ive', 'called', 'http']
['pending', 'payment', 'made', 'credit', 'card', 'received', 'confirmation', 'bank', 'yet', 'show', 'pending', 'system']
['want', 'inv

['amp', 'driver', 'nearly', 'running', 'using', 'turn', 'signal', 'http']
['u', 'people', 'respond', 'willing', 'take', 'complain', 'driver', 'taking', 'http']
['please', 'help', 'tried', 'use', 'cash', 'saturday', 'doesnt', 'seem', 'http']
['booked', 'relative', 'paidviapaytmbut', 'end', 'ride', 'driver', 'asked', 'http']
['booked', 'relative', 'paidviapaytmbut', 'end', 'ride', 'driver', 'asked', 'http']
['added', 'gift', 'card', 'today', 'use', 'added']
['last', 'year', 'made', 'approximately', 'hour', 'three', 'hour', 'hanging', 'around', 'wrong', 'part', 'town', 'http']
['im', 'diamond', 'level', 'user', 'encountering', 'issue', 'pickup', 'dropoff', 'work', 'location', 'http']
['serious', 'problem', 'recent', 'charge', 'trying', 'contact', 'support', 'http', 'tcotedopdfjje']
['well', 'iv', 'inboexed', 'number', 'picture']
['talk', 'someone', 'cant', 'log', 'account', 'anymore', 'got', 'message', 'http']
['account', 'hacked', 'havent', 'able', 'use', 'month', 'gone', 'back', 'n', 'f

...

['could', 'guy', 'help', 'day', 'without', 'response']
['take', 'type', 'malfunctioning', 'concern', 'http']
['take', 'type', 'malfunctioning', 'concern', 'http']
['take', 'type', 'malfunctioning', 'concern', 'http']
['take', 'type', 'malfunctioning', 'concern', 'http']
['hi', 'valid', 'permanent', 'resident', 'permit', 'resident', 'card', 'expired', 'waiting', 'http']
['definitely', 'want', 'get', 'sorted', 'please', 'send', 'account', 'phone', 'number', 'via', 'dm', 'assist']
['exactly', 'fix', 'look', 'like', 'resolved', 'http']
['hi', 'used', 'short', 'trip', 'took', 'le', 'ur', 'driver', 'cranky', 'http']
['hi', 'used', 'short', 'trip', 'took', 'le', 'ur', 'driver', 'cranky', 'http']
['hi', 'used', 'short', 'trip', 'took', 'le', 'ur', 'driver', 'cranky', 'http']
['hi', 'used', 'short', 'trip', 'took', 'le', 'ur', 'driver', 'cranky', 'http']
['also', 'add', 'pretty', 'sus', 'get', 'receipt', 'email', 'notification', 'http']
['also', 'add', 'pretty', 'sus', 'get', 'receipt', 'email'

['running', 'trip', 'right', 'cab', 'today', 'also', 'denied', 'http']
['running', 'trip', 'right', 'cab', 'today', 'also', 'denied', 'http']
['running', 'trip', 'right', 'cab', 'today', 'also', 'denied', 'http']
['running', 'trip', 'right', 'cab', 'today', 'also', 'denied', 'http']
['running', 'trip', 'right', 'cab', 'today', 'also', 'denied', 'http']
['running', 'trip', 'right', 'cab', 'today', 'also', 'denied', 'http']
['cab', 'available', 'nobody', 'accepting', 'ride', 'accepted', 'cancelled', 'immediatelysame', 'http']
['need', 'help', 'booked', 'ride', 'sultanpur', 'east', 'kailash', 'driver', 'ended', 'trip', 'gk', 'http']
['thinking', 'uninstall', 'http']
['thinking', 'uninstall', 'http']
['thinking', 'uninstall', 'http']
['every', 'order', 'send', 'put', 'right', 'place', 'according', 'guest', 'request', 'want', 'know', 'http', 'tcolcjpiqatnt']
['cab', 'driver', 'refused', 'switch', 'ac', 'saying', 'ac', 'need', 'repair', 'http']
['cab', 'driver', 'refused', 'switch', 'ac', 's

['pleaseeee']
['hi', 'good', 'morning', 'another', 'week', 'passedbut', 'account', 'still', 'unavailable', 'told', 'http']
['hello', 'besties', 'putting', 'thi', 'tl']
['hello', 'besties', 'putting', 'thi', 'tl']
['able', 'book', 'auto', 'ride', 'facing', 'issue', 'since', 'month']
['hello', 'im', 'courier', 'working', 'lisbon', 'portugal', 'ah', 'day', 'discovered', 'several', 'http']
['terrible', 'company', 'dont', 'even', 'speak', 'people', 'direct', 'deactivate', 'people', 'http']
['complaint', 'people', 'selling', 'app', 'mobile', 'http']
['complaint', 'people', 'selling', 'app', 'mobile', 'http']
['whats', 'taking', 'long', 'going', 'hour', 'still', 'cant', 'log', 'job', 'fix', 'account']
['team', 'located', 'information', 'touch', 'shortly', 'via', 'email', 'appreciate', 'patience']
['going', 'kolkata', 'driver', 'turning', 'ac', 'even', 'heat', 'http']
['couldnt', 'follow', 'busy', 'day', 'called', 'earlier', 'http', 'tcoedhdfmqope']
['couldnt', 'follow', 'busy', 'day', 'called

['account', 'someone', 'used', 'debit', 'card', 'purchase', 'lot', 'food', 'using', 'http']
['account', 'someone', 'used', 'debit', 'card', 'purchase', 'lot', 'food', 'using', 'http']
['dmd']
['good', 'morning', 'name', 'juan', 'antonio', 'guerrero', 'pantoja', 'mexico', 'could', 'help', 'make', 'http']
['good', 'morning', 'name', 'juan', 'antonio', 'guerrero', 'pantoja', 'mexico', 'could', 'help', 'make', 'http']
['come', 'come', 'whereever']
['overcharged', 'today', 'ride', 'saying', 'outstanding', 'amount', 'previous', 'http']
['found', 'comment', 'facebook', 'taking', 'long', 'get', 'background', 'check', 'driver', 'http']
['found', 'comment', 'facebook', 'taking', 'long', 'get', 'background', 'check', 'driver', 'http']
['trip', 'date', 'march', 'trip', 'time', 'amount', 'r', 'driver', 'prasenjit', 'issue', 'http']
['help', 'resolve', 'issue', 'please', 'send', 'u', 'dm', 'phone', 'number', 'associated', 'http', 'tcovhfopetsgi']
['thank', 'dm']
['hey', 'kumar', 'sorry', 'hear', 'tr

['problem', 'app', 'talk']
['problem', 'app', 'talk']
['thanks', 'reaching', 'appreciate', 'bringing', 'attention', 'order', 'protect', 'http']
['hi', 'dave', 'happy', 'help', 'please', 'follow', 'support', 'page', 'http', 'http']
['luck', 'resorting', 'twitter', 'saying', 'power', 'http']
['assist', 'encourage', 'continue', 'working', 'email', 'support', 'team', 'help', 'provide', 'http']
['wasnt', 'utterly', 'starving', 'ordered', 'food', 'like', 'last', 'several', 'time', 'hour', 'half', 'http']
['hi', 'becky', 'happy', 'help', 'please', 'follow', 'support', 'page', 'http', 'http']
['understand', 'frustrating', 'send', 'u', 'dm', 'phone', 'number', 'follow']
['understand', 'frustrating', 'send', 'u', 'dm', 'phone', 'number', 'follow']
['hi', 'francis', 'happy', 'help', 'please', 'follow', 'support', 'page', 'http', 'http', 'tcopdnhtkwfnv']
['hi', 'cvreeny', 'happy', 'help', 'please', 'follow', 'support', 'page', 'http', 'tcofvriuukutd', 'http']
['hi', 'cvreeny', 'happy', 'help', 'pl

In [77]:
for i in uber_df_tk.index:
    source=uber_df_tk['source'][i].lower().replace("_","")
    target=uber_df_tk['target'][i].lower().replace("_","")
    text=uber_df_tk['text'][i]
    
    while(source in text):
        index=text.index(source)
        del text[index]
    while(target in text):
        index=text.index(target)
        del text[index] 
    while("uber" in text):
        index=text.index("uber")
        del text[index] 
    while("httpubersupport" in text):
        index=text.index("httpubersupport")
        del text[index] 
    while("ubersupport" in text):
        index=text.index("ubersupport")
        del text[index] 
    while("httpubereat" in text):
        index=text.index("ubersupport")
        del text[index] 
        
    index=0
    for word in text:
        if("http" in word):
            del text[index]
        index+=1
        
    uber_df_tk['text'][i]=text

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [78]:
pickle.dump(uber_df_tk, open("./data/uber_tk.p", "wb" ) )